# Data Acquistion :

In [7]:
# Import Pandas to provide DataFrame support
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Requests
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

In [2]:

# Yaml is used to store some of the required configurations
import yaml

with open("config.yaml", "r") as f:
    cfg = yaml.load(f)
 
search_params = {
    'client_id': cfg['client_id'],
    'client_secret': cfg['client_secret'],
    'intent': 'browse',
    'limit': 50,
    'v': cfg['version']
}

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [8]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get(
    "https://foursquare.com/explore?mode=url&near=Chicago%2C%20IL%2C%20United%20States&nearGeoId=72057594042815334&q=Top%20Picks")

# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(page.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')

In [9]:
# The column names for the top venues dataframe
venue_columns = ['id', 
                 'score', 
                 'category', 
                 'name', 
                 'address',
                 'postalcode',
                 'city',
                 'href', 
                 'latitude', 
                 'longitude']

# Create the empty top venues dataframe
df_top_venues = pd.DataFrame(columns=venue_columns)

# For each venue in the BeautifulSoup HTML object
for venue in top_venues:
    
    # Extract the available attributes
    venue_name = venue.find(target="_blank").get_text()
    venue_score = venue.find(class_="venueScore positive").get_text()
    venue_cat = venue.find(class_="categoryName").get_text()
    venue_href = venue.find(class_="venueName").h2.a['href']
    venue_id = venue_href.split('/')[-1]

    if 'promotedTipId' in venue_id: 
        continue
        
    # Contruct the FourSquare venue API URL
    url ='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, 
        'EUDO0LACN3MV3DFIHGMVUMSWAILSHCBAOUVYP5EOGLXYTXTY',
       '41LFAJPS4WAQATSRKHGKOBPZRZBCUKB0TMWTR0VAJCIJJ3SD',
        '20180604')
    # Request the venue data
    result = requests.get(url).json()
        
    # Get the properly formatted address and the latitude and longitude
    venue_address = result['response']['venue']['location']['address']
    venue_postalcode = result['response']['venue']['location']['postalCode']
    venue_city = result['response']['venue']['location']['city']
    venue_latitude = result['response']['venue']['location']['lat']
    venue_longitude = result['response']['venue']['location']['lng']
    
    # Add the venue to the top venues dataframe
    df_top_venues = df_top_venues.append({'id': venue_id,
                                          'score': venue_score,
                                          'category': venue_cat,
                                          'name': venue_name,
                                          'address': venue_address,
                                          'postalcode': venue_postalcode,
                                          'city': venue_city,
                                          'href': venue_href,
                                          'latitude': venue_latitude,
                                          'longitude': venue_longitude}, ignore_index=True)

In [10]:
# The column names for the restaurants dataframe
restaurants_columns = ['id',
                       'score', 
                       'category', 
                       'categoryID', 
                       'name', 
                       'address',
                       'postalcode',
                       'city',
                       'latitude',
                       'longitude', 
                       'venue_name', 
                       'venue_latitude',
                       'venue_longitude']

# Create the empty top venues dataframe
df_restaurant = pd.DataFrame(columns=restaurants_columns)

# Create a list of all the top venue latitude and longitude
top_venue_lats = df_top_venues['latitude'].values
top_venue_lngs = df_top_venues['longitude'].values

# Create a list of all the top venue names
top_venue_names = df_top_venues['name'].values

# Iterate over each of the top venues
# The venue name, latitude and longitude are passed to the loop
for ven_name, ven_lat, ven_long in zip(top_venue_names, top_venue_lats, top_venue_lngs):
    
    # Configure additional Search parameters
    # This is the FourSquare Category Id for all food venues
    categoryId = '4d4b7105d754a06374d81259'
    radius = 500
    limit = 50
    
    # Contruct the FourSquare search API URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
       'EUDO0LACN3MV3DFIHGMVUMSWAILSHCBAOUVYP5EOGLXYTXTY',
        '41LFAJPS4WAQATSRKHGKOBPZRZBCUKB0TMWTR0VAJCIJJ3SD',
        ven_lat,
        ven_long,
        '20180604',
        categoryId,
        radius,
        limit)
    
    # Make the search request
    results = requests.get(url).json()
    
    # Want a good selection of Restaurents
    # If less than 10 are returned ignore
    if len(results['response']['venues']) < 10:
        continue
        
    # Populate the new dataframe with the list of restaurants
    # Get the values for each Restaurant from the JSON
    for restaurant in results['response']['venues']:
 
        # Sometimes the Venue JSON is missing data. If so ignore and continue
        try:
            # Get location details
            rest_id = restaurant['id']
            rest_category = restaurant['categories'][0]['pluralName']
            rest_categoryID = restaurant['categories'][0]['id']
            rest_name = restaurant['name']
            rest_address = restaurant['location']['address']
            rest_postalcode = restaurant['location']['postalCode']
            rest_city = restaurant['location']['city']
            rest_latitude = restaurant['location']['lat']
            rest_longitude = restaurant['location']['lng']
            
            # Contruct the FourSquare venue API URL to get the venues rating / score
            rest_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
                rest_id, 
                 'EUDO0LACN3MV3DFIHGMVUMSWAILSHCBAOUVYP5EOGLXYTXTY',
        '41LFAJPS4WAQATSRKHGKOBPZRZBCUKB0TMWTR0VAJCIJJ3SD',
                '20180604')

            # Get the restaurant score and href
            result = requests.get(rest_url).json()
            rest_score = result['response']['venue']['rating']
            
            # Add the restaurant details to the dataframe
            df_restaurant = df_restaurant.append({'id': rest_id,
                                                  'score': rest_score,
                                                  'category': rest_category,
                                                  'categoryID': rest_categoryID,
                                                  'name': rest_name,
                                                  'address': rest_address,
                                                  'postalcode': rest_postalcode,
                                                  'city': rest_city,
                                                  'latitude': rest_latitude,
                                                  'longitude': rest_longitude,
                                                  'venue_name': ven_name,
                                                  'venue_latitude': ven_lat,
                                                  'venue_longitude': ven_long}, ignore_index=True)
            
        # If there are any issue with a restaurant ignore and continue
        except:
            continue

In [11]:
df_restaurant.shape

(422, 13)

In [15]:
# Finally write the dataframe to a pickle file for restoring later
df_restaurant.to_pickle('.\capstone_pickle\restaurants.pkl')

In [16]:
# These are the columns that we want to keep.
# Columns not listed here won't be imported, speeding things up.
crime_keep_columns = ['CASE#',
                      'DATE  OF OCCURRENCE',
                      'BLOCK', 
                      ' PRIMARY DESCRIPTION',
                      'WARD',
                      'LATITUDE',
                      'LONGITUDE']

In [17]:
df = pd.read_csv('https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv',
                 usecols=crime_keep_columns)

In [18]:
df.shape

(260020, 7)

In [19]:
df.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,PRIMARY DESCRIPTION,WARD,LATITUDE,LONGITUDE
0,JB485663,10/20/2018 01:05:00 PM,011XX N LEAVITT ST,THEFT,2.0,NaN,NaN
1,JC355180,11/14/2018 12:01:00 AM,035XX S MICHIGAN AVE,OTHER OFFENSE,3.0,NaN,NaN
2,JB517463,11/15/2018 06:18:20 PM,011XX W 31ST ST,NARCOTICS,11.0,NaN,NaN
3,JC320782,06/24/2019 06:20:00 PM,041XX S DREXEL BLVD,THEFT,4.0,NaN,NaN
4,JC270482,05/19/2019 11:00:00 AM,018XX S WOOD ST,DECEPTIVE PRACTICE,25.0,NaN,NaN


In [20]:
df.tail()

,CASE#,DATE OF OCCURRENCE,BLOCK,PRIMARY DESCRIPTION,WARD,LATITUDE,LONGITUDE
260015,JC356571,07/19/2019 09:00:00 PM,031XX W HARRISON ST,OTHER OFFENSE,24.0,41.873699,-87.704705
260016,JC318635,06/23/2019 04:40:00 PM,005XX N CENTRAL AVE,BATTERY,37.0,41.889742,-87.765198
260017,JC405144,08/23/2019 11:00:00 PM,019XX W SCHILLER ST,THEFT,1.0,41.907177,-87.675381
260018,JC379670,08/05/2019 12:30:00 PM,074XX S HARVARD AVE,BATTERY,6.0,41.759037,-87.633299
260019,JC316226,06/18/2019 04:00:00 PM,034XX W 71ST PL,BURGLARY,17.0,41.763391,-87.708287


In [21]:
# Strip leading & trailing whitespace
df.columns = df.columns.str.strip()

# Replace multiple spaces with a single space
df.columns = df.columns.str.replace('\s{2,}', ' ')

# Replace # with blank
df.columns = df.columns.str.replace('#', '')

# Replace spaces with _
df.columns = df.columns.str.replace(' ', '_')

# Convert to lowercase
df.columns = df.columns.str.lower()

# Move September 2017 dates to September 2018
df.date_of_occurrence.replace(to_replace="(09/\\d+)/2017", value=r"\1/2018", regex=True, inplace=True)

In [22]:
# Verify that all datatype are as expected
df.dtypes

case                    object
date_of_occurrence      object
block                   object
primary_description     object
ward                   float64
latitude               float64
longitude              float64
dtype: object

In [23]:
df['date_of_occurrence'] =  pd.to_datetime(df['date_of_occurrence'], format='%m/%d/%Y %I:%M:%S %p')

In [24]:
# Add new columns to the dataframe to allow hourly, daily & monthly analysis
df['hour'] = df['date_of_occurrence'].dt.hour
df['day_name'] = df['date_of_occurrence'].dt.day_name()
df['day'] = df['date_of_occurrence'].dt.dayofweek + 1
df['month_name'] = df['date_of_occurrence'].dt.month_name()
df['month'] = df['date_of_occurrence'].dt.month
df['year'] = df['date_of_occurrence'].dt.year
df['year_month'] = df['date_of_occurrence'].dt.to_period('M')

In [25]:
# Add the zip and street attributes
df['zip'] = df.block.str.split(' ').str[0]
df['street'] = df.block.str.split(' ').str[1:].apply(', '.join)

In [26]:
# Verify that all rows have valid data
df.isna().sum()

case                      0
date_of_occurrence        0
block                     0
primary_description       0
ward                      8
latitude               2594
longitude              2594
hour                      0
day_name                  0
day                       0
month_name                0
month                     0
year                      0
year_month                0
zip                       0
street                    0
dtype: int64

In [27]:
# Drop rows with missing values 
df.dropna(inplace=True)

In [28]:
# Drop rows with missing values 
df.dropna(inplace=True)

In [29]:
# Have a final look at the crime dataframe
df.head()

,case,date_of_occurrence,block,primary_description,ward,latitude,longitude,hour,day_name,day,month_name,month,year,year_month,zip,street
11,JC320310,2019-06-24 18:24:00,077XX S PAULINA ST,BATTERY,17.0,41.753506,-87.665947,18,Monday,1,June,6,2019,2019-06,077XX,"S, PAULINA, ST"
12,JC319636,2019-06-24 11:00:00,061XX N MOZART ST,THEFT,50.0,41.992936,-87.700697,11,Monday,1,June,6,2019,2019-06,061XX,"N, MOZART, ST"
13,JC425833,2019-08-10 23:00:00,035XX N FREMONT ST,CRIMINAL DAMAGE,44.0,41.945898,-87.651858,23,Saturday,6,August,8,2019,2019-08,035XX,"N, FREMONT, ST"
14,JC427264,2019-09-05 15:00:00,017XX N AUSTIN AVE,THEFT,29.0,41.911965,-87.775314,15,Thursday,4,September,9,2019,2019-09,017XX,"N, AUSTIN, AVE"
15,JC422139,2019-09-05 17:00:00,026XX N SOUTHPORT AVE,THEFT,32.0,41.930451,-87.663499,17,Thursday,4,September,9,2019,2019-09,026XX,"N, SOUTHPORT, AVE"


In [31]:
# Finally write the dataframe to a pickle file for restoring later
df.to_pickle('.\capstone_pickle\crimes.pkl')